<a href="https://colab.research.google.com/github/rostro36/Partisan-Responses/blob/master/09_GraphWriter_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/Partisan-Responses-master')

Install all requirements.

In [ ]:
!pip install torchtext -U
!pip install spacy==2.1.0
!pip install neuralcoref allennlp hnswlib allennlp-models
!pip install wandb
!python -m spacy download en

Import everything

In [ ]:
import neuralcoref
import spacy
import re
import pickle
import pandas as pd
import gc
from Answer import Answer
import utils

Load corefferenced data

In [ ]:
df = pd.read_pickle("coref.pkl")
df.head()

,question,answer,party
0,Should abortion be illegal?,Mr. Speaker. Congress now has an opportunity t...,R
1,Should abortion be illegal?,"Mr. Speaker, I rise today to express my opposi...",R
2,Should abortion be illegal?,"so eloquently pointed out earlier, partialbirt...",R
3,Should abortion be illegal?,"Mr. Chairman, I rise in reluctant support of t...",D
4,Should abortion be illegal?,Will the Senator yield to answer a question so...,D


Setup for parsing

In [ ]:
identifier='oie'
last_check=1
step_size=100
#df is still from the cell above above

def parse_entry(entry,verb_dict,verb_list):
    result=dict()
    result['question']=' '.join([token.text for token in utils.sp(entry['question'])])
    phrase_corpus, triplet_id, parsed_text,parsed=Answer(entry['answer']).create_test(verb_dict,verb_list)
    result['corpus']=' ; '.join(phrase_corpus)
    result['tags']=' '.join(['<phrase>']*len(phrase_corpus))
    result['triplet_id']=' ; '.join([re.sub('\,','',str(x))[1:-1] for x in triplet_id])
    result['parsed_text']=parsed_text
    result['parsed']=' '.join([str(x) for x in parsed])
    return result

def load_pickles(identifier, checkpoint):
  if os.path.exists("verb_dict"+identifier+str(checkpoint)+".pickle"):
    with open('verb_dict'+identifier+str(checkpoint)+'.pickle', 'rb') as handle:
      verb_dict = pickle.load(handle)
    with open('verb_list'+identifier+str(checkpoint)+'.pickle', 'rb') as handle:
      verb_list = pickle.load(handle)
    with open('result'+identifier+str(checkpoint)+'.pickle', 'rb') as handle:
      result = pickle.load(handle)
  else:
    print('File does not exist yet.')
    verb_dict=dict()
    verb_list=[]
    result=[]
  return verb_dict, verb_list, result

def dump_pickles(identifier, checkpoint, verb_dict, verb_list, result):
  with open("verb_dict"+identifier+str(checkpoint)+".pickle", 'wb') as handle:
      pickle.dump(verb_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
  with open("verb_list"+identifier+str(checkpoint)+".pickle", 'wb') as handle:
      pickle.dump(verb_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
  with open("result"+identifier+str(checkpoint)+".pickle", 'wb') as handle:
      pickle.dump(result, handle, protocol=pickle.HIGHEST_PROTOCOL)
  return 

verb_dict = pickle.load(open("verb_dict.pickle", "rb"))
verb_list = None

Process data

In [ ]:
result = []
df = df[df['answer'].apply(lambda x: len(x)) < 1000]
df.apply(lambda x: result.append(parse_entry(x,verb_dict,verb_list)), axis=1)
dump_pickles(identifier,1+df.shape[0]/step_size,verb_dict,verb_list,result)

done with ordered
DO
DO
DO
BELLYACHE
DO
DISFAVOR
DO
DO
GENDERBASE
DO
GENDERBASE
DISCUSS
DENOTE
DO
DO
DO
DO
DO
COOK
COOK
DO
INFECT
DO


Post-processing of data

In [ ]:
df=pd.DataFrame(result) 
df=df.sample(frac=1, random_state=36)
df=df.dropna(axis=0, how='any')

Write to variables

In [ ]:
identifier='../GraphWriter-master/data/preprocessed'
df.to_csv("/content/drive/My Drive/GraphWriter-master/data/preprocessed.test.tsv", sep='\t', index=False, header=False)

Use file with lowest vloss in the folder "partisanResponses" of GraphWriter-master to generate responses on test data.


E.g. in the next cell "9.vloss-3.980539.lr-0.1" was used.

In [ ]:
os.chdir('/content/drive/My Drive/GraphWriter-master')
!python generator.py -save=partisan-responses/first_20_ckpt/9.vloss-3.980539.lr-0.1

2020-07-24 20:13:56.955420: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Loading Data from  data/preprocessed.train.tsv
data/
preprocessed.train.tsv
building vocab
done
Vocab sizes:
src 6847
ent 18458
nerd 4
rel 18429
out 9801
graph
0 42
/content/drive/My Drive/GraphWriter-master/models/newmodel.py:152: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cx = torch.tensor(hx)
That is correct . What we are trying to do is restate in simple , easy to understand language THE BALANCE that THE COURTS , I think , have properly struck between <unk> THE PURPOSE OF THE 21ST AMENDMENT and at the same time making sure that WE do not subtract in any way FROM THE INTERSTATE COMME

In [ ]:
with open("/content/drive/My Drive/GraphWriter-master/outputs/9.vloss-3.980539.lr-0.1.inputs.beam_predictions.cmdline") as f:
  gw_gen_answers = f.readlines()

In [ ]:
df['GW_ANS'] = gw_gen_answers

In [ ]:
pickle.dump(df, open("GW_output_200724.pkl", "wb"))

In [ ]:
pickle.load(open("GW_output_200724.pkl", "rb"))

,question,corpus,tags,triplet_id,parsed_text,parsed,GW_ANS
28,What do you think about the current president ?,the courts ; the balance ; the purpose of the ...,<phrase> <phrase> <phrase> <phrase> <phrase> <...,0 252 1 ; 0 500 2 ; 3 7406 4 ; 3 1388 5 ; 3 12...,That is correct . What we are trying to do is ...,6 -1 0 1 7 2 8 3 4 9 5 10 11 6 -1,"mr. chairman , if the gentleman will yield , w..."
24,Should every American have equal opportunities...,by Madam Speaker and the House Republican lead...,<phrase> <phrase> <phrase> <phrase>,0 905 1 ; 1 2039 2 ; 1 8310 1 ; 1 58 3,"Madam Speaker , <phrase_1> have been assured <...",0 1 5 4 -1 2 6 7 4 -1 4 -1 4 -1 3 8 4 -1 4 -1 ...,"mr. president , i that i have to say that i ha..."
11,What do you think about the current president ?,Mr. Chairman ; my time ; we all ; with ; citiz...,<phrase> <phrase> <phrase> <phrase> <phrase> <...,0 3932 1 ; 2 285 3 ; 4 5158 5 ; 2 285 6,"<phrase_0> , reclaiming <phrase_1> for a momen...",0 1 8 7 -1 7 -1 7 -1 2 3 9 4 5 10 7 -1 6 11 7 -1,"mr. chairman , if the gentleman will yield fur..."
39,What is the biggest threat to America ?,I ; Senator LIEBERMAN ; a question ; by the Ir...,<phrase> <phrase> <phrase> <phrase> <phrase>,0 4201 1 ; 0 1309 2 ; 3 4790 4,"Mr. President , <phrase_0> see <phrase_1> . Se...",0 1 6 5 -1 5 -1 2 7 5 -1 3 4 8 5 -1 5 -1,"mr. president , i that i me say to the senator..."
7,Should we have higher taxes for higher incomes ?,"the Democrats ; the Tax Code ; share""-I ; that...",<phrase> <phrase> <phrase> <phrase> <phrase> <...,0 82 1 ; 0 237 1 ; 2 15 3 ; 3 5158 1 ; 1 565 3...,"So when the Democrats , in 1993 , said . "" <ph...",7 -1 0 1 8 9 2 3 10 11 12 4 5 13 14 7 -1 6 15 ...,"if the senator will allow that , let me say to..."
12,What do you think about the current president ?,the courts ; the balance ; the purpose of the ...,<phrase> <phrase> <phrase> <phrase> <phrase> <...,0 252 1 ; 0 500 2 ; 3 7406 4 ; 3 1388 5 ; 3 12...,That is correct . What we are trying to do is ...,6 -1 0 1 7 2 8 3 4 9 5 10 11 6 -1,"mr. chairman , if the gentleman will yield , w..."
2,What do you believe about tax increases ?,the House ; the tax cut and spending cut bill ...,<phrase> <phrase> <phrase> <phrase> <phrase> <...,0 340 1 ; 2 7405 3 ; 3 1522 4 ; 5 3863 5 ; 6 3...,"Mr. Speaker , as <phrase_0> considers <phrase_...",0 1 12 2 3 13 4 14 11 -1 11 -1 5 15 11 -1 11 -...,"mr. president , i am not a member of the commi..."
40,What is the biggest threat to America ?,this body ; quite a while ; the same amount of...,<phrase> <phrase> <phrase> <phrase> <phrase> <...,0 362 1 ; 0 485 2 ; 3 4918 4 ; 4 3955 5,intended to offer this amendment in order to r...,6 -1 0 1 7 6 -1 2 8 6 -1 3 4 9 5 10 6 -1,"mr. president , i that i thought it would be a..."
10,Should we have universal healthcare ?,Mr. Speaker ; for universal healthcare ; H. Re...,<phrase> <phrase> <phrase> <phrase> <phrase>,0 1380 1 ; 0 386 2 ; 0 4918 0 ; 0 4918 3 ; 4 5...,"Mr. Speaker , today Mr. Speaker stand for a co...",0 1 6 2 7 5 -1 5 -1 8 5 -1 3 9 5 -1 5 -1 4 10 ...,"mr. president , mr. speaker that i know of , i..."
17,Should we reduce national debt ?,Mr. President ; no commitment,<phrase> <phrase>,0 4201 1,"Mr. President , the Senator from Wyoming talke...",2 -1 2 -1 2 -1 0 1 3 2 -1,that is the reason mr. president on the floor ...
